# Local Food Wastage Management System — In-Memory SQLite
Runs entirely in RAM using `sqlite3.connect(':memory:')`. No DB installation or file required.

**Datasets:** `providers_data.csv`, `receivers_data.csv`, `food_listings_data.csv`, `claims_data.csv`

**What you get:** 17 SQL analyses, CRUD demos, and simple EDA plots (matplotlib only).

### Setup

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

PROVIDERS_CSV = r"/mnt/data/providers_data.csv"
RECEIVERS_CSV = r"/mnt/data/receivers_data.csv"
FOOD_LISTINGS_CSV = r"/mnt/data/food_listings_data.csv"
CLAIMS_CSV = r"/mnt/data/claims_data.csv"
print("CSV paths set.")

### Load & Clean

In [ ]:
providers = pd.read_csv(PROVIDERS_CSV)
receivers = pd.read_csv(RECEIVERS_CSV)
food = pd.read_csv(FOOD_LISTINGS_CSV)
claims = pd.read_csv(CLAIMS_CSV)

def norm(df):
    df.columns = [c.strip().replace(' ', '_') for c in df.columns]
    return df

providers, receivers, food, claims = map(norm, (providers, receivers, food, claims))

for col in ['Provider_ID','Receiver_ID','Food_ID','Claim_ID']:
    for df in [providers, receivers, food, claims]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

if 'Expiry_Date' in food.columns:
    food['Expiry_Date'] = pd.to_datetime(food['Expiry_Date'], errors='coerce')
if 'Timestamp' in claims.columns:
    claims['Timestamp'] = pd.to_datetime(claims['Timestamp'], errors='coerce')

for df in [providers, receivers, food, claims]:
    for c in df.columns:
        if df[c].dtype == 'O':
            df[c] = df[c].fillna('Unknown')

print("Data loaded and cleaned.")

### Create In-Memory SQLite DB

In [ ]:
con = sqlite3.connect(':memory:')
cur = con.cursor()

cur.executescript('''
CREATE TABLE providers(
  Provider_ID INTEGER PRIMARY KEY,
  Name TEXT, Type TEXT, Address TEXT, City TEXT, Contact TEXT
);
CREATE TABLE receivers(
  Receiver_ID INTEGER PRIMARY KEY,
  Name TEXT, Type TEXT, City TEXT, Contact TEXT
);
CREATE TABLE food_listings(
  Food_ID INTEGER PRIMARY KEY,
  Food_Name TEXT, Quantity INTEGER, Expiry_Date TEXT,
  Provider_ID INTEGER, Provider_Type TEXT, Location TEXT, Food_Type TEXT, Meal_Type TEXT,
  FOREIGN KEY(Provider_ID) REFERENCES providers(Provider_ID)
);
CREATE TABLE claims(
  Claim_ID INTEGER PRIMARY KEY,
  Food_ID INTEGER, Receiver_ID INTEGER, Status TEXT, Timestamp TEXT,
  FOREIGN KEY(Food_ID) REFERENCES food_listings(Food_ID),
  FOREIGN KEY(Receiver_ID) REFERENCES receivers(Receiver_ID)
);
''')

food_sql = food.copy()
if 'Expiry_Date' in food_sql.columns:
    food_sql['Expiry_Date'] = food_sql['Expiry_Date'].astype(str)
claims_sql = claims.copy()
if 'Timestamp' in claims_sql.columns:
    claims_sql['Timestamp'] = claims_sql['Timestamp'].astype(str)

providers.to_sql('providers', con, if_exists='append', index=False)
receivers.to_sql('receivers', con, if_exists='append', index=False)
food_sql.to_sql('food_listings', con, if_exists='append', index=False)
claims_sql.to_sql('claims', con, if_exists='append', index=False)

print("In-memory DB ready.")

### Helper

In [ ]:
import pandas as pd
def run(sql): return pd.read_sql_query(sql, con)
print("run(sql) ready.")

## Analyses (17 SQL queries)

**Providers & Receivers by city**

In [ ]:
run('''SELECT city, COUNT(DISTINCT Provider_ID) AS providers,
 (SELECT COUNT(DISTINCT Receiver_ID) FROM receivers r WHERE r.city=p.city) AS receivers
 FROM providers p GROUP BY city ORDER BY providers DESC, receivers DESC;''')

**Provider type by total quantity**

In [ ]:
run('''SELECT Provider_Type, SUM(COALESCE(Quantity,0)) AS total_quantity FROM food_listings GROUP BY Provider_Type ORDER BY total_quantity DESC;''')

**Provider contacts in a city**

In [ ]:
run('''SELECT Name, Type, Address, City, Contact FROM providers WHERE LOWER(City)=LOWER('Chennai');''')

**Receivers with most claims**

In [ ]:
run('''SELECT r.Receiver_ID, r.Name, r.Type, r.City, COUNT(c.Claim_ID) AS total_claims FROM receivers r JOIN claims c ON c.Receiver_ID=r.Receiver_ID GROUP BY r.Receiver_ID, r.Name, r.Type, r.City ORDER BY total_claims DESC;''')

**Total quantity available**

In [ ]:
run('''SELECT SUM(COALESCE(Quantity,0)) AS total_quantity_available FROM food_listings;''')

**City with most listings**

In [ ]:
run('''SELECT Location AS City, COUNT(*) AS listings_count FROM food_listings GROUP BY Location ORDER BY listings_count DESC;''')

**Most common food types**

In [ ]:
run('''SELECT Food_Type, COUNT(*) AS freq FROM food_listings GROUP BY Food_Type ORDER BY freq DESC;''')

**Claims per food item**

In [ ]:
run('''SELECT f.Food_ID, f.Food_Name, COUNT(c.Claim_ID) AS claim_count FROM food_listings f LEFT JOIN claims c ON c.Food_ID=f.Food_ID GROUP BY f.Food_ID, f.Food_Name ORDER BY claim_count DESC;''')

**Provider with most completed claims**

In [ ]:
run('''SELECT p.Provider_ID, p.Name, COUNT(c.Claim_ID) AS completed_claims FROM providers p JOIN food_listings f ON f.Provider_ID=p.Provider_ID JOIN claims c ON c.Food_ID=f.Food_ID WHERE LOWER(c.Status)='completed' GROUP BY p.Provider_ID, p.Name ORDER BY completed_claims DESC;''')

**Claim status distribution**

In [ ]:
run('''WITH total AS (SELECT COUNT(*) AS n FROM claims) SELECT Status, COUNT(*) AS count, ROUND(100.0*COUNT(*)/(SELECT n FROM total),2) AS pct FROM claims GROUP BY Status ORDER BY count DESC;''')

**Avg quantity per claim per receiver**

In [ ]:
run('''WITH cj AS (SELECT c.Claim_ID, c.Receiver_ID, f.Quantity FROM claims c LEFT JOIN food_listings f ON f.Food_ID=c.Food_ID) SELECT r.Receiver_ID, r.Name, ROUND(AVG(COALESCE(cj.Quantity,0)),2) AS avg_quantity_per_claim FROM receivers r JOIN cj ON cj.Receiver_ID=r.Receiver_ID GROUP BY r.Receiver_ID, r.Name ORDER BY avg_quantity_per_claim DESC;''')

**Most claimed meal type**

In [ ]:
run('''SELECT f.Meal_Type, COUNT(c.Claim_ID) AS claims_count FROM food_listings f JOIN claims c ON c.Food_ID=f.Food_ID GROUP BY f.Meal_Type ORDER BY claims_count DESC;''')

**Total quantity by provider**

In [ ]:
run('''SELECT p.Provider_ID, p.Name, SUM(COALESCE(f.Quantity,0)) AS total_donated FROM providers p JOIN food_listings f ON f.Provider_ID=p.Provider_ID GROUP BY p.Provider_ID, p.Name ORDER BY total_donated DESC;''')

**Listings expiring in 48 hours**

In [ ]:
run('''SELECT Food_ID, Food_Name, Quantity, Expiry_Date, Location, Provider_ID FROM food_listings WHERE Expiry_Date IS NOT NULL AND datetime(Expiry_Date) <= datetime('now','+2 days') ORDER BY datetime(Expiry_Date);''')

**Unclaimed listings**

In [ ]:
run('''SELECT f.Food_ID, f.Food_Name, f.Quantity, f.Location, f.Expiry_Date FROM food_listings f LEFT JOIN claims c ON c.Food_ID=f.Food_ID WHERE c.Claim_ID IS NULL ORDER BY f.Expiry_Date;''')

**Top cities by completed claims**

In [ ]:
run('''SELECT f.Location AS City, COUNT(c.Claim_ID) AS completed_claims FROM claims c JOIN food_listings f ON f.Food_ID=c.Food_ID WHERE LOWER(c.Status)='completed' GROUP BY f.Location ORDER BY completed_claims DESC;''')

**Avg claim timestamp (julian day) by city**

In [ ]:
run('''SELECT f.Location AS City, ROUND(AVG(julianday(c.Timestamp)),2) AS avg_claim_time_jd FROM claims c JOIN food_listings f ON f.Food_ID=c.Food_ID WHERE c.Timestamp IS NOT NULL GROUP BY f.Location ORDER BY avg_claim_time_jd;''')

## CRUD examples on `food_listings`

In [ ]:
# CREATE
cur.execute("""INSERT INTO food_listings(Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type)
VALUES (999997, 'Sample Idli', 25, date('now','+1 day'), (SELECT Provider_ID FROM providers LIMIT 1), 'Restaurant', 'SampleCity', 'Vegetarian', 'Breakfast')""")
con.commit()
import pandas as pd; pd.read_sql_query("SELECT * FROM food_listings WHERE Food_ID=999997", con)

In [ ]:
# READ
import pandas as pd; pd.read_sql_query("SELECT Food_ID, Food_Name, Quantity, Location FROM food_listings ORDER BY Food_ID DESC LIMIT 5", con)

In [ ]:
# UPDATE
cur.execute("UPDATE food_listings SET Quantity=Quantity+5 WHERE Food_ID=999997")
con.commit()
import pandas as pd; pd.read_sql_query("SELECT Food_ID, Food_Name, Quantity FROM food_listings WHERE Food_ID=999997", con)

In [ ]:
# DELETE
cur.execute("DELETE FROM food_listings WHERE Food_ID=999997")
con.commit()
import pandas as pd; pd.read_sql_query("SELECT * FROM food_listings WHERE Food_ID=999997", con)

## EDA (matplotlib only)

In [ ]:
# Top cities by listings
import pandas as pd, matplotlib.pyplot as plt
city = pd.read_sql_query("SELECT Location AS City, COUNT(*) AS Listings FROM food_listings GROUP BY Location ORDER BY Listings DESC LIMIT 10", con)
plt.figure(); plt.bar(city['City'].astype(str), city['Listings']); plt.title('Top 10 Cities by Food Listings'); plt.xticks(rotation=45, ha='right'); plt.ylabel('Listings'); plt.tight_layout(); plt.show()

In [ ]:
# Quantity by provider type
ptype = pd.read_sql_query("SELECT Provider_Type, SUM(COALESCE(Quantity,0)) AS Total FROM food_listings GROUP BY Provider_Type ORDER BY Total DESC", con)
plt.figure(); plt.bar(ptype['Provider_Type'].astype(str), ptype['Total']); plt.title('Total Quantity by Provider Type'); plt.xticks(rotation=45, ha='right'); plt.ylabel('Quantity'); plt.tight_layout(); plt.show()

In [ ]:
# Claim status distribution
status = pd.read_sql_query("SELECT Status, COUNT(*) AS C FROM claims GROUP BY Status", con)
plt.figure(); plt.pie(status['C'], labels=status['Status'].astype(str), autopct='%1.1f%%'); plt.title('Claim Status Distribution'); plt.show()